
# Consolidación de Algoritmos de Optimización

Este notebook contiene tres algoritmos de optimización basados en PSO (Particle Swarm Optimization):
1. **PSO Simple**: Minimización de la función \( f(x, y) = x^2 + y^2 \).
2. **PSO con PySwarm**: Minimización de la misma función utilizando la biblioteca `pyswarm`.
3. **PSO con Restricciones**: Maximización de \( f(x) = 3x1 + 5x2 \) sujeto a restricciones lineales.

Cada uno de los algoritmos ha sido implementado en funciones independientes para facilitar su uso.


In [ ]:

import numpy as np
from pyswarm import pso  # Para el PSO con pyswarm

# --------------------------------------------------------------------
# PSO SIMPLE: Minimización de f(x, y) = x^2 + y^2
# --------------------------------------------------------------------
def pso_simple(funcion_objetivo, num_particulas=20, dim=2, iteraciones=10, c1=2.0, c2=2.0, w=0.7, limite_inf=-100, limite_sup=100):
    # Inicialización
    particulas = np.random.uniform(limite_inf, limite_sup, (num_particulas, dim))
    velocidades = np.zeros((num_particulas, dim))
    pbest = particulas.copy()
    fitness_pbest = np.array([funcion_objetivo(p[0], p[1]) for p in particulas])
    gbest = pbest[np.argmin(fitness_pbest)]
    fitness_gbest = np.min(fitness_pbest)

    # Búsqueda iterativa
    for iteracion in range(iteraciones):
        for i in range(num_particulas):
            r1, r2 = np.random.rand(), np.random.rand()
            velocidades[i] = w * velocidades[i] + c1 * r1 * (pbest[i] - particulas[i]) + c2 * r2 * (gbest - particulas[i])
            particulas[i] += velocidades[i]
            particulas[i] = np.clip(particulas[i], limite_inf, limite_sup)
            fitness = funcion_objetivo(particulas[i][0], particulas[i][1])

            # Actualización de pbest y gbest
            if fitness < fitness_pbest[i]:
                fitness_pbest[i] = fitness
                pbest[i] = particulas[i].copy()
                if fitness < fitness_gbest:
                    fitness_gbest = fitness
                    gbest = particulas[i].copy()

    return gbest, fitness_gbest

# --------------------------------------------------------------------
# PSO con PySwarm: Minimización de f(x, y) = x^2 + y^2 utilizando la biblioteca pyswarm
# --------------------------------------------------------------------
def pso_pyswarm(funcion_objetivo, limites_inf, limites_sup, num_particulas=10, iteraciones=20):
    solucion_optima, valor_optimo = pso(funcion_objetivo, limites_inf, limites_sup, swarmsize=num_particulas, maxiter=iteraciones, debug=False)
    return solucion_optima, valor_optimo

# --------------------------------------------------------------------
# PSO con Restricciones
# --------------------------------------------------------------------
def pso_restricciones(funcion_objetivo, restricciones, n_particulas=20, n_dimensiones=2, max_iteraciones=100, c1=2, c2=2, w=0.5):
    # Inicialización de partículas y matrices
    x = np.zeros((n_particulas, n_dimensiones))
    v = np.zeros((n_particulas, n_dimensiones))
    pbest = np.zeros((n_particulas, n_dimensiones))
    pbest_fit = -np.inf * np.ones(n_particulas)
    gbest = np.zeros(n_dimensiones)
    gbest_fit = -np.inf

    # Inicialización aleatoria respetando restricciones
    for i in range(n_particulas):
        while True:
            x[i] = np.random.uniform(0, 10, n_dimensiones)
            if all(restriccion(x[i]) for restriccion in restricciones):
                break
        v[i] = np.random.uniform(-1, 1, n_dimensiones)
        pbest[i] = x[i].copy()
        fit = funcion_objetivo(x[i])
        if fit > pbest_fit[i]:
            pbest_fit[i] = fit

    # Optimización PSO con restricciones
    for _ in range(max_iteraciones):
        for i in range(n_particulas):
            fit = funcion_objetivo(x[i])
            if fit > pbest_fit[i] and all(restriccion(x[i]) for restriccion in restricciones):
                pbest_fit[i] = fit
                pbest[i] = x[i].copy()
                if fit > gbest_fit:
                    gbest_fit = fit
                    gbest = x[i].copy()

            v[i] = w * v[i] + c1 * np.random.rand() * (pbest[i] - x[i]) + c2 * np.random.rand() * (gbest - x[i])
            x[i] += v[i]
            if not all(restriccion(x[i]) for restriccion in restricciones):
                x[i] = pbest[i].copy()

    return gbest, gbest_fit


In [ ]:

# Función objetivo para los ejemplos de PSO simple y con pyswarm
def funcion_objetivo_pso(x, y=None):
    if y is None:
        return x[0]**2 + x[1]**2
    return x**2 + y**2

# Ejecutar PSO simple
print("PSO Simple:")
solucion_simple, valor_simple = pso_simple(funcion_objetivo_pso)
print(f"Solución óptima: {solucion_simple}, Valor óptimo: {valor_simple}")

# Ejecutar PSO con pyswarm
print("PSO con PySwarm:")
solucion_pyswarm, valor_pyswarm = pso_pyswarm(funcion_objetivo_pso, [-100, -100], [100, 100])
print(f"Solución óptima: {solucion_pyswarm}, Valor óptimo: {valor_pyswarm}")

# PSO con restricciones
def funcion_objetivo_restricciones(x):
    return 3 * x[0] + 5 * x[1]

# Definir restricciones para el PSO con restricciones
restricciones = [
    lambda x: x[0] - 4 <= 0,            # x1 <= 4
    lambda x: 2 * x[1] - 12 <= 0,        # 2x2 <= 12
    lambda x: 3 * x[0] + 2 * x[1] - 18 <= 0  # 3x1 + 2x2 <= 18
]

# Ejecutar PSO con restricciones
print("PSO con Restricciones:")
solucion_restric, valor_restric = pso_restricciones(funcion_objetivo_restricciones, restricciones)
print(f"Mejor solución: {solucion_restric}, Valor óptimo: {valor_restric}")
